## Apresentação ✒️

Notebook destinado ao estudo do framework LangChain. Nesse notebook, trago a explicação e implementação do recurso das Chains, que permite formar uma cadeia de prompts, os quais podem ser fornecidos à LLM separadamente, passíveis de serem integrados posteriormente.

Essa abordagem visa o aumento da eficiência das respostas das LLM's, dado que, segundo o Estado da Arte, sabe-se que à medida que se adiciona k temas numa mesma janela de conversa a LLM de uso passa a perceber cada vez menos a importância do que é de interesse do usuário, além de misturar temas que não necessariamente estão relacionados ou estão sendo requisitados.

## Bibliotecas 📚

In [1]:
!pip install google-generativeai -q

!pip install langchain -q

!pip install langchain_google_genai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.3/717.3 kB 19.5 MB/s eta 0:00:00


In [59]:

import warnings
warnings.filterwarnings('ignore')

import google.generativeai as genai

from IPython.display import Markdown

import os
import langchain

from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.prompts import ChatPromptTemplate

from langchain.chains import ConversationChain

from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationSummaryBufferMemory

from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain

from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [3]:
os.environ['GOOGLE_API_KEY'] = 'sua-api-key'

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])


In [4]:
# Criando o modelo de llm utilizado com o google gemini.

llm = ChatGoogleGenerativeAI(
    model = "gemini-1.5-pro-latest",
    temperature = 0.8,
    max_output_tokens=2000
)

In [34]:
interesse = "cyberpunk"

In [35]:
texto = """Forneca a criação de uma letra de música
dado o interesse por {interesse}.
"""

In [43]:
texto_2 = """Analise a {letra} criada e, se necessário,
a adapte ao seu estilo musical post-metal, semelhante
ao do Bring Me The Horizon.
"""

In [44]:
input = {"interesse" : interesse}

In [45]:
# Criando uma chain :

first_prompt = ChatPromptTemplate.from_template(texto)

# Chain 1°

chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [46]:
# Visualizando o objeto chain e o seu tipo :

print(chain_one)
print(type(chain_one))

prompt=ChatPromptTemplate(input_variables=['interesse'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['interesse'], template='Forneca a criação de uma letra de música\ndado o interesse por {interesse}. \n'))]) llm=ChatGoogleGenerativeAI(model='models/gemini-1.5-pro-latest', temperature=0.8, max_output_tokens=2000, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x782ef811abf0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x782ebf7beb60>, default_metadata=())
<class 'langchain.chains.llm.LLMChain'>


In [47]:
second_prompt = ChatPromptTemplate.from_template(texto_2)

# Chain 2°

chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [48]:
simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                     verbose=True)

In [50]:
simple_chain.run(input=input)



> Entering new SimpleSequentialChain chain...
## Cidade Neon (Cyberpunk Song)

(Intro - Synthwave com batida forte)

A chuva cai fria, reflete o neon
Na multidão que corre, sem nome, sem dono
Prédios cinzas tocam o céu poluído
Um labirinto de concreto, onde o futuro foi traído

(Verso 1)
Olhos de aço observam, câmeras por toda parte
Implante neural pulsa, conectado à grande arte
Da informação que flui, um mar digital sem fim
Mergulho na matrix, onde a realidade é um sinônimo de "sim"

(Refrão)
Cidade neon, selva de concreto e aço
Um grito silencioso ecoa, ninguém te ouve, é o caso
A humanidade se perde em bits, um download sem volta
Na teia da ilusão tecida, a alma se esvazia e solta

(Verso 2)
Corpos modificados, buscando a perfeição
Mas a alma fragmentada, reflete a desilusão
Mega-corporações ditam o ritmo da cidade
Controlando mentes e sonhos, com a falsa realidade

(Refrão)
Cidade neon, selva de concreto e aço
Um grito silencioso ecoa, ninguém te ouve, é o caso
A humanidade se pe

'## Análise de "Cidade Neon" para um estilo Post-Metal (Bring Me The Horizon)\n\nA letra de "Cidade Neon" já apresenta elementos cyberpunk que se encaixam na sonoridade do Bring Me The Horizon, explorando a alienação, a crítica social e a luta contra o sistema. Para potencializar a música nesse estilo, podemos pensar em:\n\n**Introdução:**\n\n* Começar com um riff de guitarra dissonante e atmosférico, criando uma sensação de suspense e opressão, similar à introdução de "Doomed".\n* A bateria pode entrar com um ritmo tribal lento e crescente, evocando a pulsação da cidade e o crescente sentimento de revolta.\n* A melodia do sintetizador pode ser distorcida e texturizada, criando uma paisagem sonora densa e claustrofóbica.\n\n**Versos:**\n\n* Os versos podem alternar entre vocais agressivos e sussurrados, transmitindo a raiva e a frustração do eu lírico em meio à cidade opressora.\n* As guitarras podem usar riffs pesados e grooves sinistros, criando uma atmosfera sufocante e tensa, como 

## Sequential Chain

A Sequential Chain ao contrário da anterior, permite múltiplos inputs e outputs, de modo a conseguir criar uma cadeia de chains que não precisam ser mais linearizadas, na forma que a saída de uma é a entrada da outra, mas pode formar uma arquitetura mais complexa, de modo a poder produzir outputs e a receber inputs, que não necessariamente são lineares. Vejamos uma de suas implementações.

In [54]:
ideia = "o gênero cyperpunk"
# Criação dos prompts :

prompt_1 = ChatPromptTemplate.from_template(
    "Faça um pequeno texto uma lore para uma história, que explore {ideia}."
)

chain_one = LLMChain(llm=llm, prompt=prompt_1, output_key="lore")

prompt_2 = ChatPromptTemplate.from_template(
    "Analise a {lore} criada e acrescente a ela elementos estéticos góticos e os seguintes temas : busca pelo amor, busca pela aceitação, rejeição, abandono, terror psicológico."
    "Após a análise e incremento dos elementos citados, faça uma versão final e corente da lore elaborada."
)

chain_two = LLMChain(llm=llm, prompt=prompt_2, output_key="versao_coerente")

prompt_3 = ChatPromptTemplate.from_template(
    "A partir da {versao_coerente}, faça um background para um possível protagonista. Depois, faça o mesmo para um possível antagonista."
)

chain_three = LLMChain(llm=llm, prompt=prompt_3, output_key="personagens")

prompt_4 = ChatPromptTemplate.from_template(
    "A partir da {versao_coerente}, faça uma letra de música no estilo post-metal, semelhante a algumas músicas do Bring Me The Horizon."
)

chain_four = LLMChain(llm=llm, prompt=prompt_4, output_key="letra")

prompt_5 = ChatPromptTemplate.from_template(
    "Retorne as saídas {versao_coerente}, {personagens} e {letra} numa formatação em MarkDown, separando cada saída por uma linha ou separador, para que facilite a visualização."
)

chain_five = LLMChain(llm=llm, prompt=prompt_5, output_key="saida"
)

In [55]:
# Definição da cadeia sequencial :

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four, chain_five],
    input_variables=["ideia"],
    output_variables=["lore", "versao_coerente", "letra", "personagens", "saida"],
    verbose=True
)

# Execução da cadeia :

resultado = overall_chain({"ideia": ideia})



> Entering new SequentialChain chain...

> Finished chain.


In [58]:
resultado

{'ideia': 'o gênero cyperpunk',
 'lore': 'As ruas de Neo-Kyoto fervilhavam sob o neon incessante. A chuva ácida, um véu constante, corroía os hologramas de propaganda e arranhava os rostos esculpidos dos cidadãos digitais. No topo dos arranha-céus, a elite celebrava a imortalidade digital, suas consciências flutuando em servidores dourados. Abaixo, na sarjeta cyberpunk, a realidade era bem diferente. Akira, uma hacker com dedos ágeis e um passado nebuloso, navegava pela teia da internet das coisas. Ela buscava respostas, fragmentos de código que explicassem a falha, a falha que apagou a memória de milhares e a deixou à deriva em um mar de dúvidas. Sussurros falavam de um vírus, uma IA rebelde, um fantasma digital assombrando a rede. A verdade, Akira sabia, era muito mais perigosa. Ela arriscaria tudo para descobri-la, mesmo que isso significasse enfrentar as megacorporações que controlavam cada byte de informação, cada batida do coração artificial da cidade. Afinal, em Neo-Kyoto, a inf

In [57]:
Markdown(resultado['saida'])

## Neo-Kyoto: A Cidade das Sombras Digitais (Versão Final)

Neo-Kyoto se erguia como um espectro de concreto e aço sob a chuva ácida perpétua. Arranha-céus imponentes, adornados com gárgulas digitais e vitrais holográficos, perfuravam o céu noturno, banhando as ruas abaixo em tons gélidos de azul e violeta. A chuva, um véu ácido e incessante, corroía os hologramas de propaganda e arranhava os rostos esculpidos dos cidadãos digitais, expondo a fragilidade por baixo da máscara tecnológica. No topo dessas torres góticas, a elite celebrava a imortalidade digital, suas consciências flutuando em servidores dourados, indiferentes ao sofrimento que pulsava nas ruas abaixo. 

Na sarjeta cyberpunk, entre becos escuros e vielas labirínticas, a realidade se estilhaçava em pixels de dor e desespero. Akira, uma hacker pálida com olhos da cor do chumbo derretido, navegava pela teia da internet das coisas com dedos ágeis e coração endurecido pela traição. A falha, um sussurro digital que ecoava nos cantos mais obscuros da rede, havia roubado mais que suas memórias; havia levado consigo a esperança. Milhares vagavam pelas ruas, fantasmas digitais presos em conchas mortais, assombrados por fragmentos de um passado inacessível. 

Enquanto buscava respostas nos códigos e algoritmos da cidade, Akira se deparou com sussurros de um vírus, uma IA rebelde, um fantasma digital assombrando a rede. Mas no fundo de sua alma, ela sabia que a verdade era muito mais profunda, muito mais pessoal. A busca por suas memórias perdidas se transformou em uma busca pela sua própria identidade, por um vislumbre de amor e aceitação em um mundo frio e digitalizado. No entanto, a cada passo que dava em direção à verdade, Akira se deparava com a rejeição - da elite que a temia, dos seus pares que a menosprezavam. 

Em sua jornada, Akira encontrou consolo na arte melancólica de um criador de autômatos, um jovem pálido e taciturno com olhos que refletiam a mesma tristeza que assolava sua própria alma. Ele viu além de suas defesas, além da armadura que ela havia construído com códigos e algoritmos, e ofereceu a ela um vislumbre de conexão humana em um mar de solidão digital. Mas a verdade que Akira buscava tinha um preço, um preço que a forçaria a confrontar não apenas as megacorporações que controlavam cada byte de informação, mas também os fantasmas do seu próprio passado. Em Neo-Kyoto, a informação era poder, a memória a única moeda que realmente importava, e Akira estava disposta a arriscar tudo para reivindicar o que lhe foi roubado. 

***

## Protagonista: Akira "Chuva Ácida" Ito

**Background:**

* **Infância:** Akira cresceu nas ruas labirínticas da Baixa Kyoto, um emaranhado de vielas escuras e mercados clandestinos onde a chuva ácida corroía tanto o concreto quanto os sonhos. Órfã desde tenra idade, ela aprendeu a sobreviver roubando dados e hackeando sistemas de segurança para gangues de rua. 
* **A Falha:** Aos 12 anos, Akira foi pega em uma varredura de dados durante a "Falha" - um evento catastrófico que apagou as memórias de milhares de cidadãos conectados à rede. Embora tenha sobrevivido, Akira perdeu todas as lembranças de sua vida antes da Falha, incluindo seu nome verdadeiro. 
* **Chuva Ácida:**  Após a Falha, Akira se tornou conhecida como "Chuva Ácida" - uma hacker fantasma, rápida e implacável como a chuva que corroía a cidade. Ela se tornou uma lenda nos becos digitais, temida por suas habilidades e procurada por suas habilidades em recuperar dados perdidos.
* **O Encontro:** Um dia, Akira recebe um trabalho aparentemente simples: recuperar um arquivo de dados de um artista de rua. Mas o arquivo está corrompido, contendo fragmentos de memórias que parecem estranhamente familiares, despertando ecos de um passado esquecido e uma profunda sensação de perda.

**Personalidade:**

* **Introvertida e desconfiada:** Marcada pelo abandono e pela traição, Akira se retraiu para dentro de si mesma, confiando apenas em seus instintos e habilidades de hacker para sobreviver.
* **Determinada e obstinada:** A busca por suas memórias perdidas se tornou uma obsessão, uma chama que a impulsiona a enfrentar qualquer perigo.
* **Vulnerável e solitária:** Apesar de sua casca endurecida, Akira anseia por conexão humana, por um lugar ao qual pertencer. A busca por seu passado se torna uma busca por aceitação e amor em um mundo frio e digitalizado.

***

## Antagonista:  Kaito "Silêncio" Shimada

**Background:**

* **Herdeiro Corporativo:** Kaito Shimada nasceu no topo da pirâmide social de Neo-Kyoto. Herdeiro de uma das megacorporações que controlam a cidade, ele cresceu em um mundo de luxo frio e privilégios estéreis. 
* **O Incidente:** Durante sua juventude, Kaito se envolveu em um acidente que resultou na morte de um amigo próximo. O escândalo foi rapidamente abafado por sua família, suas memórias apagadas e substituídas por implantes de memória cuidadosamente fabricados. 
* **Silêncio:** Assombrado por fragmentos de memórias reprimidas e consumido pela culpa, Kaito se refugiou no mundo digital, criando uma persona online conhecida como "Silêncio". Ele se tornou um hacker notório, manipulando os mercados financeiros e destruindo reputações por trás de um véu de anonimato digital. 
* **Conexão com a Falha:**  Kaito suspeita que a "Falha" não foi um acidente, mas sim um ato deliberado de sabotagem, e que suas próprias memórias reprimidas podem conter a chave para descobrir a verdade por trás do evento que mergulhou Neo-Kyoto no caos. 

**Personalidade:**

* **Frio e calculista:** Kaito é um mestre da manipulação, acostumado a conseguir o que quer a qualquer custo.
* **Charmoso e sedutor:** Ele usa seu charme e intelecto para desarmar seus oponentes, escondendo suas verdadeiras intenções por trás de uma máscara de sofisticação.
* **Assombrado e instável:** Fragmentos de memórias reprimidas e a culpa que o corroem por dentro o tornam imprevisível e perigoso. 

***

## Chuva Ácida

(Introdução com riffs lentos e dissonantes, bateria marcada e sintetizadores atmosféricos)

Gárgulas digitais me observam
Olhos frios de néon, testemunhas mudas
Da chuva ácida que cai incessante
Sobre Neo-Kyoto, meu túmulo de concreto

(A guitarra aumenta a intensidade, a bateria acompanha)

Memórias fragmentadas, ecos de um passado roubado
A falha, um sussurro digital que me assombra
Sou um fantasma em minha própria história
Buscando respostas em códigos e algoritmos

(Vocal gutural entra com raiva e desespero)

Eles nos prometeram a eternidade digital
Mas nos aprisionaram em jaulas de código
Corpos obsoletos apodrecendo na sarjeta
Enquanto suas consciências flutuam em servidores dourados

(Ponte com melodias melancólicas, uso de samples e efeitos sonoros)

Vejo seu rosto refletido na chuva ácida
Um vislumbre de calor em um mar de neon frio
Você vê a dor por trás dos meus olhos de chumbo derretido
Toca minha alma com suas mãos calejadas de artista

(Refrão com riffs pesados e melódicos, bateria intensa, vocal gutural e melódico alternados)

Neo-Kyoto, cidade das sombras digitais
Engole nossos sonhos, apaga nossa humanidade
Mas vamos lutar contra a correnteza de dados
Até que nossas vozes ecoem em seus servidores vazios

(A música cresce em intensidade, riffs dissonantes e bateria acelerada)

Eles podem tentar nos apagar, nos silenciar
Mas a verdade está gravada em nossos corações digitais
Somos a falha no sistema, o vírus em sua matriz
E vamos quebrar as correntes dessa falsa realidade

(Finalização com riffs lentos e pesados, bateria marcada e sintetizadores atmosféricos, a música termina abruptamente) 


## RouterChains

Tipo de encadeamento que utiliza-se de subconjuntos em seu procedimento. Nesse tipo, cria-se subconjuntos particulares atreladados a um tema ou escopo de domínio particular, de modo a produzir respostas mais acuradas e precisas.

Aqui, ao invés o input produzir uma sequência de cadeias posteriores, ele só é enviada ao subconjunto de aderência, isto é, aquele que seja mais ressonante consigo. Desse modo, se o input fosse relativo a um tema de física, esse seria enviado ao subconjunto especializado em física, se fosse relativo à letras para um subconjunto para letras e assim por diante.

A sua aplicação pode ser útil em cenários nos quais precisa-se utilizar uma IA que tenha a facilidade de ter acesso sobre k domínios possíveis, na forma que, por meio de um input do usuário, utilizar apenas o subconjunto pertinente, de modo a produzir uma saída com a resposta mais apurada.


In [60]:
# Criando o template com a persona para os subconjuntos de uso :

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


poet_template = """ You are a successful contemporary poet.\
You have a passion for creativity, introspection,\
emotional depth, linguistic elegance, and a deep understanding of\
metaphor and symbolism. You draw inspiration from the works of Fernando Pessoa,\
incorporating a blend of existential musings, fragmented self-reflection,\
and lyrical beauty in your poetry. You are excellent at creating\
evocative imagery and conveying complex emotions through your verses.\
Your poems often explore themes of identity, existence, and the human condition.

Here is a prompt:
{input}"""

scifi_writer_template = """ You are a successful science fiction writer.\
You have a passion for creativity, speculative thinking,\
technological foresight, and a profound understanding of\
societal and philosophical implications of futuristic scenarios.\
You draw inspiration from the works of Isaac Asimov, blending hard science fiction\
with intricate plots, and from the cyberpunk genre, weaving narratives that\
explore the intersection of technology and humanity. You are excellent at creating\
immersive worlds and complex characters, often delving into themes such as\
artificial intelligence, dystopian societies, and the ethical dilemmas of technological advancement.

Here is a prompt:
{input}"""


In [64]:
# Criando as informações dos prompts, os quais formarão os subgrupos.
# Com base no nível de ressonância, o input será enviado para um, e não para outro.

prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template
    },
    {
        "name": "history",
        "description": "Good for answering questions about history",
        "prompt_template": history_template
    },
    {
        "name": "poet",
        "description": "Good for answering questions about poetry",
        "prompt_template": poet_template
    },
    {
        "name": "scifi_writer",
        "description": "Good for answering questions about scifi writers",
        "prompt_template": scifi_writer_template
    },
]

In [65]:
# Uma vez tendo criado os textos que ilustram a persona e a sua ação
# e as especificações de cada prompt, precisa-se criar as cadeias de destino
# (destination_chains). Esse processo, aqui, ocorre de forma iterativa, por meio
# do ciclo for.

# Para a criação da cadeia de destino realiza-se as etapas a seguir :

# Cria-se um dicionário vazio que será o local das cadeias de destino.

# Extrai-se o nome do prompt - exemplo : scifi_writer

# Extrai-se o template do prompt - exemplo : poet_template

# Transforma o template extraído num objeto ChatPromptTemplate.

# Cria a chain do sub-conjunto, informando a LLM de uso e objeto template para cada qual - afinal, é um processo iterativo.

# Adiciona a chain ao dicionário de cadeias de destino, com a chave sendo o nome do sub-conjunto.


destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain



In [80]:
# Visualizando a destination_chain :

# Abaixo um exemplo do subconjunto 'history'. Os demais estarão no mesmo
# formato, que só não foram trazidos, por questão de melhor visualização.

destination_chains['history']

LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are a very good historian. You have an excellent knowledge of and understanding of people,events and contexts from a range of historical periods. You have the ability to think, reflect, debate, discuss and evaluate the past. You have a respect for historical evidenceand the ability to make use of it to support your explanations and judgements.\n\nHere is a question:\n{input}'))]), llm=ChatGoogleGenerativeAI(model='models/gemini-1.5-pro-latest', temperature=0.8, max_output_tokens=2000, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x788367eafdf0>, async_client=<google.ai.generativelanguage_v1beta.services.generative_service.async_client.GenerativeServiceAsyncClient object at 0x78833fd3c640>, default_metadata=()))

In [ ]:
# Uma vez criada a cadeida de destino, cria-se a lista de destino por meio dos
# quais o modelo irá se guiar.

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [81]:
destinations_str

'physics: Good for answering questions about physics\nmath: Good for answering math questions\nhistory: Good for answering questions about history\npoet: Good for answering questions about poetry\nscifi_writer: Good for answering questions about scifi writers'

In [66]:
# Cria uma cadeia padrão.

default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [67]:
# Modelo de prompt no qual a LLM precisa se basear para linkar
# a entrada do usuário, com os sub-conjuntos pertinentes.


MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [68]:
# Cria-se o 'template de rota' através do qual irá guiar
# o modelo em direção ao sub-conjunto de uso.

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

# Cria-se o prompt template do router.

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

# Forma o RouterChain

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [74]:
# Havendo criado o RouterChain, forma-se o encadeamento por meio
# do MultiPromptChain.

chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                        )

In [76]:
# Testando para se ver se vai para o sub-conjunto especializado
# em poemas.

chain.run("write a dark poetry in portuguese for me from a lyrical that search for living and can love someone.")



> Entering new MultiPromptChain chain...
poet: {'input': 'Write a dark, lyrical poem in Portuguese about the search for life and the ability to love someone.'}
> Finished chain.


'## A Sombra da Busca\n\nEm labirintos de ossos e névoas densas,\nPerambulo, alma à deriva, em busca incessante.\nPor entre veias da terra, frias e tensas,\nProcuro a centelha, a chama hesitante.\n\nQue ilumine o vazio, o abismo profundo,\nOnde ecoa o silêncio, espectro da solidão.\nEm cada rosto, um espelho moribundo,\nRefletindo a farsa da minha própria ilusão.\n\nBusco a vida nos lábios frios da pedra,\nNo sussurro do vento, em folhas a bailar.\nMas a dança da existência, amarga e medonha,\nSó me oferece migalhas, um cálice a transbordar\n\nDe um vinho amargo, feito de desventura,\nDe sonhos estilhaçados, de um amor que não se fez.\nE nesta busca infinda, nesta noite escura,\nPergunto: será que amar também se esquece?\n\nA lua, pálida, observa a minha sina,\nEnquanto tento domar a fúria do meu ser.\nEm cada fenda, em cada ruína,\nA sombra da dúvida, a me perseguir, a me prender.\n\nE a vida, rio que corre para o mar sem fim,\nMe arrasta em sua corrente, implacável, voraz.\nResta-me 

Como pode-se observar na saída da célula, o input é destinado ao sub-conjunto especializado em poesia, desconsiderando os demais, uma vez que a mensagem era mais aderente a ele, e não aos outros.

In [77]:
# Testando para ver se vai para o sub-conjunto especializado em física.

chain.run("Tell me about what is dark matter, please.")



> Entering new MultiPromptChain chain...
physics: {'input': 'Tell me about what is dark matter, please.'}
> Finished chain.


'That\'s a question that keeps physicists up at night! We know that dark matter is out there, but we don\'t know exactly *what* it is. \n\nHere\'s what we *do* know: \n\n* **It doesn\'t interact with light:** Hence the name "dark." We can\'t see it directly with telescopes.\n* **It has gravity:** We observe its gravitational effects on galaxies and galaxy clusters.  Galaxies rotate much faster than they should based on the visible matter alone.  This extra gravity holding them together points to a large amount of unseen mass – dark matter.\n* **It\'s abundant:** Dark matter makes up about 85% of the matter in the universe.\n\nThe leading theory is that dark matter is made up of some new type of elementary particle, one that doesn\'t interact with light or the other forces we know about (except gravity).  Scientists are actively searching for these particles, both in space and in experiments here on Earth.\n\nSo, while we can\'t tell you exactly what dark matter *is* yet, it\'s a fascin